In [13]:
import pickle
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack

In [3]:
# load models from file
def load_all_models(n_models, model_dir):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = model_dir + '/model_' + str(i + 1) + '.nn'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('loaded %s' % filename)
	return all_models

In [4]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

In [5]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# fit standalone model
	model = LogisticRegression()
	model.fit(stackedX, inputy)
	return model

In [6]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

In [7]:
# load all models
n_members = 3
members = load_all_models(n_members, 'test_models')
print('Loaded %d models' % len(members))

loaded test_models/model_1.nn
loaded test_models/model_2.nn
loaded test_models/model_3.nn
Loaded 3 models


In [27]:
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)
test = test.fillna(0)
x_train = train[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_train = np.asarray(train.iloc[:,-8:])
x_test = test[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_test = np.asarray(test.iloc[:,-8:])
scaler = MinMaxScaler() #scale features between 0 and 1
x_train = scaler.fit_transform(x_train)
x_train = np.asarray(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = scaler.fit_transform(x_test)
x_test = np.asarray(x_test).reshape(x_test.shape[0], 1, x_test.shape[1])

In [16]:
# evaluate standalone models on test dataset
for model in members:
	_, acc = model.evaluate(x_test, y_test, verbose=0)
	print('Model Accuracy: %.3f' % acc)


Model Accuracy: 0.546
Model Accuracy: 0.542
Model Accuracy: 0.579


In [29]:
# fit stacked model using the ensemble
y_train_decode = np.argmax(y_train, axis=1)
model = fit_stacked_model(members, x_train, y_train_decode)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [33]:
# evaluate model on test set
yhat = stacked_prediction(members, model, x_test)
acc = accuracy_score(y_test_decode, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

Stacked Test Accuracy: 0.580
